In [319]:
import copy, datetime, glob
from dateutil import parser
import ACILib

InterestQueueName = ACILib.ACI_InterestQueueName
###### Path in Container setup ######
flask_container_path = ''
notebook_container_path = '/home/jovyan'
current_container_path = notebook_container_path
logfilename = current_container_path + '/HTTSDashboard/logs/RabbitMQ_ACI_Event.log*'
logfilelist = glob.glob(logfilename)

event_path = '/HTTSDashboard/logs/ACI/events/'
acceptfilename = current_container_path + event_path + '*_AcceptEvent.txt'
acceptfilelist = glob.glob(acceptfilename)
inqueuefilename = current_container_path + event_path + '*_InQueueEvent.txt'
inqueuefilelist = glob.glob(inqueuefilename)

AcceptEventsFromFile = ACILib.FindAcceptEventsFromEventFile(acceptfilelist)
InQueueEventsFromFile = ACILib.FindInQueueEventsFromEventFile(inqueuefilelist)
######Return is by year######
InQueueEventsFromFile[2020][-10:]

['689106118-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-23 05:26:07',
 '689141613-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-23 05:30:20',
 '689117227-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-23 05:30:22',
 '689149779-~1-~FTS-~2020-05-24 00:01:50',
 '689129275-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-24 00:08:04',
 '689149814-~4-~CX-APJC-SYD-ACI-SSPT-~2020-05-24 00:18:28',
 '689055780-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-24 00:33:24',
 '689146885-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-24 00:45:38',
 '689108265-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-24 01:00:12',
 '689140240-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-24 01:00:14']

In [329]:
InQueueEventsFromFile[2020][-10:]

def ConvertInQueueEventsByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),InQueueEventsFromFile={}):    
    InQueueEventsByDate = {} # {'2020-05-22,{'CX-APJC-BLR-ACI-SSPT':[689148383],'CX-APJC-SYD-ACI-SSPT':[689148487]}}
    for year,events in InQueueEventsFromFile.items():
        if year < parser.parse(start_date).year:
            continue
        elif year > parser.parse(end_date).year:
            break
        for event in events:
            caseno,casesev,queue,timestamp = event.split('-~')
            #print(" {} - {}".format(timestamp,end_date))
            if parser.parse(timestamp) < parser.parse(start_date):
                continue
            elif parser.parse(timestamp) >= parser.parse(end_date)+datetime.timedelta(days=1):
                break
            date = parser.parse(timestamp).strftime("%Y-%m-%d")
            if date not in InQueueEventsByDate.keys():
                InQueueEventsByDate[date] = {}
            if queue not in InQueueEventsByDate[date]:
                InQueueEventsByDate[date][queue] = []
            InQueueEventsByDate[date][queue].append(caseno)
    return InQueueEventsByDate

InQueueEventsByDate = ConvertInQueueEventsByDate(start_date='2020-02-27',InQueueEventsFromFile=InQueueEventsFromFile)
InQueueEventsByDate['2020-05-24']

{'FTS': ['689149779'],
 'CX-APJC-BLR-ACI-SSPT': ['689129275', '689055780', '689146885', '689108265'],
 'CX-APJC-SYD-ACI-SSPT': ['689149814', '689140240']}

In [342]:
AcceptEventsByDate = {} # {'2020-05-22',{caseno:[[zdazhi,tetration],[zmeng,HTTS]]}}

def ConvertAcceptEventsByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),AcceptEventsFromFile=[]):
    AcceptEventsByDate = {}
    for year,events in AcceptEventsFromFile.items():
        if year < parser.parse(start_date).year:
            continue
        elif year > parser.parse(end_date).year:
            break
        for event in events:
            caseno,casesev,ccoid,workgroup,timestamp = event.split('-~')
            if parser.parse(timestamp) < parser.parse(start_date):
                continue
            elif parser.parse(timestamp) > parser.parse(end_date)+datetime.timedelta(days=1):
                break
            date = parser.parse(timestamp).strftime("%Y-%m-%d")
            if date not in AcceptEventsByDate.keys():
                AcceptEventsByDate[date] = {}
            if caseno not in AcceptEventsByDate[date].keys():
                AcceptEventsByDate[date][caseno] = []
            if ccoid != 'lowtouch':
                AcceptEventsByDate[date][caseno].append([ccoid,workgroup])
        return AcceptEventsByDate
    
RawAcceptEventsByDate = ConvertAcceptEventsByDate(start_date='2020-02-27',AcceptEventsFromFile=AcceptEventsFromFile) 
RawAcceptEventsByDate['2020-05-23']

{'689111638': [['roagraw2', 'UC_WORKGROUP']],
 '689136652': [['minkwong', 'APT-ACI-SOLUTIONS']],
 '689146430': [['siddhp', 'APT-ACI-SOLUTIONS']],
 '689148383': [['kahande', 'APT-ACI-SOLUTIONS2']],
 '689148462': [['maveer', 'APT-ACI-SOLUTIONS2']],
 '689027437': [['hethakur', 'APT-ACI-SOLUTIONS2']],
 '689148487': [['siddhp', 'APT-ACI-SOLUTIONS']],
 '689148561': [['deepaky', 'APT-ACI-SOLUTIONS2']],
 '689148432': [['raghb', 'APT-ACI-SOLUTIONS2']],
 '689148585': [['roagraw2', 'APT-ACI-SOLUTIONS2']],
 '689123995': [['rajbg', 'UC_WORKGROUP']],
 '689117227': [['kahande', 'APT-ACI-SOLUTIONS2']],
 '689141613': [['minkwong', 'APT-ACI-SOLUTIONS']],
 '689106118': [['kahande', 'APT-ACI-SOLUTIONS2']]}

In [344]:
###### For FTS_Workgroup and UC_Workgroup Find previous accepted case to identify the workgroup, by the reverse date #######
def FindEngrWorkgroup(date,ccoid,AcceptEventsByDate):
    end_datetime = parser.parse('2020-02-27')
    cur_datetime = parser.parse(date)
    while cur_datetime > end_datetime:
        cur_date = cur_datetime.strftime("%Y-%m-%d")
        for caseno, caestakerlist in AcceptEventsByDate[cur_date].items():
            for casetaker in caestakerlist:
                if ccoid == casetaker[0] and casetaker[1] != 'UC_WORKGROUP' and casetaker[1] != 'FTS_WORKGROUP':
                    #print("Found {} in workgroup {} on date {} from case {}".format(ccoid,casetaker[1],cur_date,caseno))
                    return ccoid, casetaker[1], cur_date
        cur_datetime = cur_datetime + datetime.timedelta(days=-1)
    return ccoid, "Unknown",date
FindEngrWorkgroup('2020-05-22','gauvasud',RawAcceptEventsByDate)

('gauvasud', 'APT-ACI-SOLUTIONS2', '2020-05-22')

In [350]:
###### 1. Only CaseInQueue and engineer takes it will take into account 2. Only APT-ACI-SOLUTIONS/2 yanked case(not InQueue) will take into account ######
def RemoveNotInQueueAcceptCase(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),RawAcceptEventsByDate=[],InQueueEventsByDate=[],InterestWorkgroup=['APT-ACI-SOLUTIONS','APT-ACI-SOLUTIONS2']):
    
    RemoveNotInQueueAcceptEventsByDate = copy.deepcopy(RawAcceptEventsByDate)
    for date,events in RawAcceptEventsByDate.items():
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
        InQueueCaseList = []
        RemoveCaseList = []
        for casetype,caselist in InQueueEventsByDate[date].items():
                InQueueCaseList = [*InQueueCaseList,*caselist]
        ###### Remove the case which is not in queue but taken and keep workgroup engineer yanked case by InterestWorkgroup######
        for caseno,casetakerlist in RawAcceptEventsByDate[date].items():
            #if date == '2020-05-24':
            #    print("{} {} {} {}".format(date,caseno,casetakerlist,InQueueCaseList))
            if caseno not in InQueueCaseList:
                ###### If there is a yank case which is not in queue, but yanked by APT-ACI-SOLUTIONS/2 engineer, it wont be removed ######.
                isYankedCase = False 
                for casetaker in casetakerlist:
                    if casetaker[1] in InterestWorkgroup:
                        #print("{} {} is a yanked case, keep in the accepted list".format(date,caseno))
                        isYankedCase = True
                if not isYankedCase:
                    #print("{} Removing the case {} as it is not the InQueue case".format(date,caseno))
                    RemoveCaseList.append(caseno)
        #if date == '2020-05-24':
            #print("Remove Case List {}".format(RemoveCaseList))
        for caseno in RemoveCaseList:
            del RemoveNotInQueueAcceptEventsByDate[date][caseno]
    
    
    UpdWGAcceptEventsByDate = copy.deepcopy(RemoveNotInQueueAcceptEventsByDate)
    #print("After Remove Case List {}".format(UpdWGAcceptEventsByDate['2020-05-24']))
    ###### Check UC_WORKGROUP and FTS_WORKGROUP Put in the right workgroup #######
    for date,caselist in RemoveNotInQueueAcceptEventsByDate.items():
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
        for caseno,casetakerlist in caselist.items():
            for idx,casetaker in enumerate(casetakerlist):
                if casetaker[1] not in InterestWorkgroup:
                    ccoid,workgroup,date_wg = FindEngrWorkgroup(date,casetaker[0],RawAcceptEventsByDate)
                    if workgroup != "Unknown":
                        #print("caseno {} {} {} {}".format(caseno,ccoid,workgroup,date_wg))
                        UpdWGAcceptEventsByDate[date][caseno][idx][1] = workgroup
    #print("Return Accept case list by date {}".format(UpdWGAcceptEventsByDate['2020-05-24']))
    return UpdWGAcceptEventsByDate

date = '2020-05-24'
UpdWGAcceptEventsByDate = RemoveNotInQueueAcceptCase(start_date='2020-02-27',RawAcceptEventsByDate=RawAcceptEventsByDate,InQueueEventsByDate=InQueueEventsByDate)
UpdWGAcceptEventsByDate['2020-05-22'], len(UpdWGAcceptEventsByDate[date])

({'689133999': [['kahande', 'APT-ACI-SOLUTIONS2']],
  '689128378': [['vanakkad', 'US-ACI-SOL-WEST'],
   ['gauvasud', 'APT-ACI-SOLUTIONS2']],
  '689140726': [['wilchong', 'APT-ACI-SOLUTIONS']],
  '689110886': [['tonzeng', 'APT-ACI-SOLUTIONS']],
  '689137507': [['prpratee', 'APT-ACI-SOLUTIONS2']],
  '689124387': [['zmeng', 'CX-APJC-SYD-DC-HTEC']],
  '689139346': [['junwa', 'APT-ACI-SOLUTIONS']],
  '689140386': [['reperuma', 'APT-ACI-SOLUTIONS2']],
  '689139008': [['deepaky', 'APT-ACI-SOLUTIONS2']],
  '689115289': [['hethakur', 'APT-ACI-SOLUTIONS2']],
  '689134216': [['siddhp', 'APT-ACI-SOLUTIONS']],
  '689141863': [['debabbar', 'APT-ACI-SOLUTIONS']],
  '689142124': [['jawalia', 'APT-ACI-SOLUTIONS2']],
  '688924443': [['maveer', 'APT-ACI-SOLUTIONS2']],
  '689100426': [['maveer', 'APT-ACI-SOLUTIONS2'],
   ['kahande', 'APT-ACI-SOLUTIONS2']],
  '689108672': [['gauvasud', 'APT-ACI-SOLUTIONS2']],
  '689142289': [['junwa', 'APT-ACI-SOLUTIONS']],
  '688998242': [['debabbar', 'APT-ACI-SOLUTIONS']

In [353]:
###### Taken Case By Group ######
def CaseTakenByEngrGrpByDate(start_date='2020-02-27',end_date=datetime.datetime.today().strftime("%Y-%m-%d"),UpdWGAcceptEventsByDate={}):
    
    CaseTakenByEngrByDate = {} # {'2020-50-23':{'APT-ACI-SOLUTIONS':{'siddhp':[689146430,689148487]}}}
    for date, items in UpdWGAcceptEventsByDate.items():
        if parser.parse(date) < parser.parse(start_date):
            continue
        elif parser.parse(date) > parser.parse(end_date):
            break
        #print("Processing date {}".format(date))
        if date not in CaseTakenByEngrByDate.keys():
            CaseTakenByEngrByDate[date] = {}
        for caseno, casetakerlist in items.items():
            for casetaker in casetakerlist:
                if casetaker[1] not in CaseTakenByEngrByDate[date].keys():
                    CaseTakenByEngrByDate[date][casetaker[1]] = {}
                if casetaker[0] not in CaseTakenByEngrByDate[date][casetaker[1]].keys():
                    CaseTakenByEngrByDate[date][casetaker[1]][casetaker[0]] = []
                CaseTakenByEngrByDate[date][casetaker[1]][casetaker[0]].append(caseno)
    return CaseTakenByEngrByDate

CaseTakenByEngrByDate = CaseTakenByEngrGrpByDate(start_date='2020-02-27',UpdWGAcceptEventsByDate=UpdWGAcceptEventsByDate)
CaseTakenByEngrByDate['2020-05-22']

{'APT-ACI-SOLUTIONS2': {'kahande': ['689133999', '689100426'],
  'gauvasud': ['689128378', '689108672'],
  'prpratee': ['689137507'],
  'reperuma': ['689140386'],
  'deepaky': ['689139008'],
  'hethakur': ['689115289'],
  'jawalia': ['689142124', '689142489'],
  'maveer': ['688924443', '689100426', '689142571']},
 'US-ACI-SOL-WEST': {'vanakkad': ['689128378']},
 'APT-ACI-SOLUTIONS': {'wilchong': ['689140726'],
  'tonzeng': ['689110886', '689142516'],
  'junwa': ['689139346', '689142289'],
  'siddhp': ['689134216'],
  'debabbar': ['689141863', '688998242']},
 'CX-APJC-SYD-DC-HTEC': {'zmeng': ['689124387']}}

Found gauvasud in workgroup APT-ACI-SOLUTIONS2 on date 2020-05-22 from case 689108672


('gauvasud', 'APT-ACI-SOLUTIONS2', '2020-05-22')

In [45]:
###### Case Volume in Total ######
GetTotCase = lambda date,InQueueEventsByDate : sum([len(caselist) for casetype,caselist in InQueueEventsByDate[date].items()])
GetTotCase('2020-05-23',InQueueEventsByDate)

13